In [ ]:
import pandas as pd
import numpy as np
import os
import glob
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr, spearmanr, wilcoxon, ttest_rel
from statsmodels.stats.diagnostic import acorr_ljungbox
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Attention
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import seaborn as sns

# Fixar seeds
np.random.seed(42)
tf.random.set_seed(42)


# ============================================================================
# MÉTRICAS AVANÇADAS
# ============================================================================

def calculate_comprehensive_metrics(y_true, y_pred, scaler=None):
    """
    Calcula múltiplas métricas de avaliação
    """
    if len(y_true) == 0 or len(y_pred) == 0:
        return {}
    
    # Desnormalizar se necessário
    if scaler is not None:
        y_true_denorm = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
        y_pred_denorm = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    else:
        y_true_denorm = y_true
        y_pred_denorm = y_pred
    
    metrics = {}
    
    # Métricas básicas
    metrics['RMSE'] = np.sqrt(mean_squared_error(y_true_denorm, y_pred_denorm))
    metrics['MAE'] = mean_absolute_error(y_true_denorm, y_pred_denorm)
    
    # MAPE (evitar divisão por zero)
    mask = y_true_denorm != 0
    if np.sum(mask) > 0:
        metrics['MAPE'] = np.mean(np.abs((y_true_denorm[mask] - y_pred_denorm[mask]) / y_true_denorm[mask])) * 100
    else:
        metrics['MAPE'] = float('inf')
    
    metrics['R2'] = r2_score(y_true_denorm, y_pred_denorm)
    
    # Correlações
    try:
        metrics['Pearson_Corr'], metrics['Pearson_P'] = pearsonr(y_true_denorm, y_pred_denorm)
        metrics['Spearman_Corr'], metrics['Spearman_P'] = spearmanr(y_true_denorm, y_pred_denorm)
    except:
        metrics['Pearson_Corr'] = 0
        metrics['Pearson_P'] = 1
        metrics['Spearman_Corr'] = 0
        metrics['Spearman_P'] = 1
    
    # Métricas de direção (importante para séries temporais!)
    if len(y_true_denorm) > 1:
        true_direction = np.diff(y_true_denorm) > 0
        pred_direction = np.diff(y_pred_denorm) > 0
        metrics['Direction_Accuracy'] = np.mean(true_direction == pred_direction) * 100
    else:
        metrics['Direction_Accuracy'] = 0
    
    # Análise de resíduos
    residuals = y_true_denorm - y_pred_denorm
    metrics['Mean_Residual'] = np.mean(residuals)
    metrics['Std_Residual'] = np.std(residuals)
    metrics['Max_Error'] = np.max(np.abs(residuals))
    metrics['Min_Error'] = np.min(np.abs(residuals))
    
    # Percentis de erro
    abs_errors = np.abs(residuals)
    metrics['Error_P50'] = np.percentile(abs_errors, 50)
    metrics['Error_P90'] = np.percentile(abs_errors, 90)
    metrics['Error_P95'] = np.percentile(abs_errors, 95)
    
    return metrics


def analyze_temporal_stability(y_true, y_pred, scaler=None, window_size=50):
    """
    Analisa estabilidade da predição ao longo do tempo
    """
    if len(y_true) < window_size:
        return {}
    
    # Desnormalizar
    if scaler is not None:
        y_true = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
        y_pred = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    
    n_windows = len(y_true) // window_size
    
    rmse_per_window = []
    mae_per_window = []
    r2_per_window = []
    
    for i in range(n_windows):
        start = i * window_size
        end = start + window_size
        
        window_true = y_true[start:end]
        window_pred = y_pred[start:end]
        
        rmse_per_window.append(np.sqrt(mean_squared_error(window_true, window_pred)))
        mae_per_window.append(mean_absolute_error(window_true, window_pred))
        r2_per_window.append(r2_score(window_true, window_pred))
    
    return {
        'RMSE_windows': rmse_per_window,
        'MAE_windows': mae_per_window,
        'R2_windows': r2_per_window,
        'RMSE_std': np.std(rmse_per_window),
        'MAE_std': np.std(mae_per_window),
        'R2_mean': np.mean(r2_per_window),
        'RMSE_trend': np.polyfit(range(len(rmse_per_window)), rmse_per_window, 1)[0] if len(rmse_per_window) > 1 else 0
    }


def analyze_by_value_regime(y_true, y_pred, scaler=None):
    """
    Analisa performance em diferentes regimes (baixo, médio, alto)
    """
    if scaler is not None:
        y_true_denorm = scaler.inverse_transform(y_true.reshape(-1, 1)).flatten()
        y_pred_denorm = scaler.inverse_transform(y_pred.reshape(-1, 1)).flatten()
    else:
        y_true_denorm = y_true
        y_pred_denorm = y_pred
    
    # Definir quartis
    q25 = np.percentile(y_true_denorm, 25)
    q75 = np.percentile(y_true_denorm, 75)
    
    # Classificar
    low_mask = y_true_denorm < q25
    mid_mask = (y_true_denorm >= q25) & (y_true_denorm <= q75)
    high_mask = y_true_denorm > q75
    
    results = {}
    
    for name, mask in [('Low', low_mask), ('Mid', mid_mask), ('High', high_mask)]:
        if np.sum(mask) > 0:
            results[name] = {
                'RMSE': np.sqrt(mean_squared_error(y_true_denorm[mask], y_pred_denorm[mask])),
                'MAE': mean_absolute_error(y_true_denorm[mask], y_pred_denorm[mask]),
                'R2': r2_score(y_true_denorm[mask], y_pred_denorm[mask]),
                'Count': np.sum(mask),
                'Mean_True': np.mean(y_true_denorm[mask])
            }
    
    return results


def analyze_residual_autocorrelation(residuals, lags=20):
    """
    Analisa autocorrelação dos resíduos (deve ser baixa para boa predição)
    """
    if len(residuals) < lags + 1:
        return {}
    
    try:
        lb_test = acorr_ljungbox(residuals, lags=lags, return_df=True)
        
        # Resíduos independentes se maioria dos p-values > 0.05
        independence_score = (lb_test['lb_pvalue'] > 0.05).mean()
        
        return {
            'ljung_box_pvalues': lb_test['lb_pvalue'].values,
            'independence_score': independence_score,
            'residuals_independent': independence_score > 0.8
        }
    except:
        return {}


def compare_predictions_statistically(errors_imputed, errors_non_imputed):
    """
    Testa significância estatística da diferença
    """
    if len(errors_imputed) == 0 or len(errors_non_imputed) == 0:
        return {}
    
    # Usar valores absolutos dos erros
    abs_errors_imp = np.abs(errors_imputed)
    abs_errors_non = np.abs(errors_non_imputed)
    
    try:
        # Teste t pareado
        t_stat, t_pvalue = ttest_rel(abs_errors_imp, abs_errors_non)
        
        # Wilcoxon (não paramétrico - mais robusto)
        w_stat, w_pvalue = wilcoxon(abs_errors_imp, abs_errors_non)
        
        return {
            't_statistic': t_stat,
            't_pvalue': t_pvalue,
            'wilcoxon_statistic': w_stat,
            'wilcoxon_pvalue': w_pvalue,
            'significantly_better': w_pvalue < 0.05 and np.mean(abs_errors_imp) < np.mean(abs_errors_non)
        }
    except:
        return {}


# ============================================================================
# PREPARAÇÃO DE DADOS - MULTIVARIADO
# ============================================================================

def prepare_multivariate_sequences(data, n_steps, feature_cols, target_col='Vazao_BBR'):
    """
    Prepara sequências MULTIVARIADAS
    """
    # Verificar se todas as colunas existem
    available_cols = [col for col in feature_cols if col in data.columns]
    if len(available_cols) == 0:
        return np.array([]), np.array([]), None, None, []
    
    # Usar múltiplas features
    feature_data = data[available_cols].values
    target_data = data[[target_col]].values
    
    # Normalizar
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    scaled_features = scaler_X.fit_transform(feature_data)
    scaled_target = scaler_y.fit_transform(target_data)
    
    X, y = [], []
    indices = []
    
    for i in range(n_steps, len(scaled_features)):
        X.append(scaled_features[i-n_steps:i, :])
        y.append(scaled_target[i, 0])
        indices.append(i)
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y, scaler_X, scaler_y, indices


def prepare_multivariate_non_imputed(data, n_steps, feature_cols, target_col='Vazao_BBR'):
    """
    Prepara sequências multivariadas removendo valores -1
    """
    # Filtrar linhas onde target != -1
    valid_data = data[data[target_col] != -1].copy()
    
    if len(valid_data) == 0:
        return np.array([]), np.array([]), None, None, []
    
    # Verificar colunas disponíveis
    available_cols = [col for col in feature_cols if col in valid_data.columns]
    if len(available_cols) == 0:
        return np.array([]), np.array([]), None, None, []
    
    feature_data = valid_data[available_cols].values
    target_data = valid_data[[target_col]].values
    
    # Normalizar
    scaler_X = MinMaxScaler()
    scaler_y = MinMaxScaler()
    
    scaled_features = scaler_X.fit_transform(feature_data)
    scaled_target = scaler_y.fit_transform(target_data)
    
    X, y = [], []
    indices = []
    
    for i in range(n_steps, len(scaled_features)):
        X.append(scaled_features[i-n_steps:i, :])
        y.append(scaled_target[i, 0])
        indices.append(i)
    
    if len(X) == 0:
        return np.array([]), np.array([]), scaler_X, scaler_y, []
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y, scaler_X, scaler_y, indices


# ============================================================================
# MODELOS MELHORADOS
# ============================================================================

def create_lstm_with_attention(input_shape, units, dropout, learning_rate):
    """
    LSTM com camada de atenção
    """
    inputs = keras.Input(shape=input_shape)
    
    # Primeira camada LSTM
    x = LSTM(units, return_sequences=True)(inputs)
    x = Dropout(dropout)(x)
    
    # Segunda camada LSTM com atenção
    lstm_out = LSTM(units // 2, return_sequences=True)(x)
    
    # Mecanismo de atenção simples
    attention = Dense(1, activation='tanh')(lstm_out)
    attention = tf.nn.softmax(attention, axis=1)
    context = tf.reduce_sum(lstm_out * attention, axis=1)
    
    # Camadas densas
    x = Dense(units // 4, activation='relu')(context)
    x = Dropout(dropout)(x)
    outputs = Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    
    return model


def create_gru_with_attention(input_shape, units, dropout, learning_rate):
    """
    GRU com camada de atenção
    """
    inputs = keras.Input(shape=input_shape)
    
    x = GRU(units, return_sequences=True)(inputs)
    x = Dropout(dropout)(x)
    
    gru_out = GRU(units // 2, return_sequences=True)(x)
    
    # Atenção
    attention = Dense(1, activation='tanh')(gru_out)
    attention = tf.nn.softmax(attention, axis=1)
    context = tf.reduce_sum(gru_out * attention, axis=1)
    
    x = Dense(units // 4, activation='relu')(context)
    x = Dropout(dropout)(x)
    outputs = Dense(1)(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
    
    return model


def create_ensemble_models(input_shape, units, dropout, learning_rate, model_type='lstm'):
    """
    Cria múltiplos modelos para ensemble
    """
    models = []
    
    for i in range(3):  # 3 modelos para ensemble
        if model_type == 'lstm':
            if i == 0:
                model = create_lstm_with_attention(input_shape, units, dropout, learning_rate)
            else:
                # Variação nos parâmetros
                varied_units = units + (i * 16)
                model = Sequential([
                    LSTM(varied_units, activation='relu', return_sequences=True, input_shape=input_shape),
                    Dropout(dropout),
                    LSTM(varied_units // 2, activation='relu'),
                    Dropout(dropout),
                    Dense(varied_units // 4, activation='relu'),
                    Dense(1)
                ])
                model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
        else:  # GRU
            if i == 0:
                model = create_gru_with_attention(input_shape, units, dropout, learning_rate)
            else:
                varied_units = units + (i * 16)
                model = Sequential([
                    GRU(varied_units, activation='relu', return_sequences=True, input_shape=input_shape),
                    Dropout(dropout),
                    GRU(varied_units // 2, activation='relu'),
                    Dropout(dropout),
                    Dense(varied_units // 4, activation='relu'),
                    Dense(1)
                ])
                model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse', metrics=['mae'])
        
        models.append(model)
    
    return models


# ============================================================================
# TREINAMENTO E AVALIAÇÃO MELHORADOS
# ============================================================================

def train_ensemble_and_evaluate(X_train, y_train, X_test, y_test, model_type, params, scaler_y):
    """
    Treina ensemble de modelos e avalia
    """
    if len(X_train) == 0 or len(X_test) == 0:
        return {}, None
    
    input_shape = (X_train.shape[1], X_train.shape[2])
    
    # Criar ensemble
    models = create_ensemble_models(input_shape, params['units'], params['dropout'], 
                                    params['learning_rate'], model_type)
    
    early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
    
    # Treinar cada modelo
    all_predictions = []
    
    for i, model in enumerate(models):
        model.fit(
            X_train, y_train,
            epochs=params['epochs'],
            batch_size=params['batch_size'],
            validation_split=0.2,
            callbacks=[early_stop],
            verbose=0
        )
        
        pred = model.predict(X_test, verbose=0).flatten()
        all_predictions.append(pred)
    
    # Ensemble: média das predições
    ensemble_pred = np.mean(all_predictions, axis=0)
    
    # Calcular métricas
    metrics = calculate_comprehensive_metrics(y_test, ensemble_pred, scaler_y)
    
    # Análises adicionais
    temporal_analysis = analyze_temporal_stability(y_test, ensemble_pred, scaler_y)
    regime_analysis = analyze_by_value_regime(y_test, ensemble_pred, scaler_y)
    
    # Análise de resíduos
    y_test_denorm = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()
    pred_denorm = scaler_y.inverse_transform(ensemble_pred.reshape(-1, 1)).flatten()
    residuals = y_test_denorm - pred_denorm
    autocorr_analysis = analyze_residual_autocorrelation(residuals)
    
    # Combinar resultados
    full_metrics = {
        **metrics,
        'temporal_stability': temporal_analysis,
        'regime_analysis': regime_analysis,
        'autocorr_analysis': autocorr_analysis
    }
    
    return full_metrics, models


# ============================================================================
# FUNÇÕES PRINCIPAIS
# ============================================================================

def find_matching_files(imputed_folder, non_imputed_folder):
    """Encontra arquivos correspondentes"""
    imputed_files = glob.glob(os.path.join(imputed_folder, "*_stacking.csv"))
    non_imputed_files = glob.glob(os.path.join(non_imputed_folder, "*_merged.csv"))
    
    non_imputed_dict = {}
    for file_path in non_imputed_files:
        filename = os.path.basename(file_path)
        link = filename.replace('_merged.csv', '')
        non_imputed_dict[link] = file_path
    
    matched_files = {}
    for imputed_file in imputed_files:
        filename = os.path.basename(imputed_file)
        link = filename.replace('_stacking.csv', '')
        
        if link in non_imputed_dict:
            matched_files[link] = {
                'imputed': imputed_file,
                'non_imputed': non_imputed_dict[link]
            }
    
    print(f"✓ Encontrados {len(matched_files)} pares de arquivos")
    return matched_files


def find_best_params_advanced(imputed_file, n_steps, param_grid, use_multivariate=True):
    """
    Busca melhores parâmetros com grid search reduzido
    """
    print("  Buscando melhores hiperparâmetros...")
    
    df = pd.read_csv(imputed_file)
    df['Data'] = pd.to_datetime(df['Data'])
    df = df.sort_values('Data').reset_index(drop=True)
    
    # Tentar multivariado
    if use_multivariate:
        feature_cols = ['Vazao_BBR', 'Atraso(ms)', 'Hop_count', 'Bottleneck']
        X, y, scaler_X, scaler_y, indices = prepare_multivariate_sequences(df, n_steps, feature_cols)
    
    # Fallback para univariado
    if not use_multivariate or len(X) == 0:
        vazao_data = df[['Vazao_BBR']].values
        scaler_y = MinMaxScaler()
        scaled_data = scaler_y.fit_transform(vazao_data)
        
        X, y = [], []
        for i in range(n_steps, len(scaled_data)):
            X.append(scaled_data[i-n_steps:i, 0])
            y.append(scaled_data[i, 0])
        
        X = np.array(X).reshape((len(X), n_steps, 1))
        y = np.array(y)
    
    if len(X) == 0:
        return None
    
    # Split
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    best_params = None
    best_score = float('inf')
    
    # Grid search simplificado
    for units in param_grid['units']:
        for dropout in param_grid['dropout']:
            for lr in param_grid['learning_rate']:
                params = {
                    'units': units,
                    'dropout': dropout,
                    'learning_rate': lr,
                    'batch_size': 32,
                    'epochs': 50
                }
                
                try:
                    metrics, _ = train_ensemble_and_evaluate(
                        X_train, y_train, X_test, y_test, 'lstm', params, scaler_y
                    )
                    
                    # Usar combinação de métricas
                    score = metrics.get('RMSE', float('inf')) + (1 - metrics.get('R2', 0))
                    
                    if score < best_score:
                        best_score = score
                        best_params = params
                        print(f"    ✓ Melhor: RMSE={metrics.get('RMSE', 0):.4f}, R2={metrics.get('R2', 0):.4f}")
                except:
                    continue
    
    return best_params


def evaluate_advanced(file_path, n_steps, model_type, params, is_imputed, use_multivariate=True):
    """
    Avaliação avançada com múltiplas métricas
    """
    df = pd.read_csv(file_path)
    df['Data'] = pd.to_datetime(df['Data'])
    df = df.sort_values('Data').reset_index(drop=True)
    
    # Preparar dados
    if use_multivariate:
        feature_cols = ['Vazao_BBR', 'Atraso(ms)', 'Hop_count', 'Bottleneck']
        if is_imputed:
            X, y, scaler_X, scaler_y, indices = prepare_multivariate_sequences(df, n_steps, feature_cols)
        else:
            X, y, scaler_X, scaler_y, indices = prepare_multivariate_non_imputed(df, n_steps, feature_cols)
    
    # Fallback univariado
    if not use_multivariate or len(X) == 0:
        if is_imputed:
            vazao_data = df[['Vazao_BBR']].values
        else:
            valid_df = df[df['Vazao_BBR'] != -1]
            vazao_data = valid_df[['Vazao_BBR']].values
        
        scaler_y = MinMaxScaler()
        scaled_data = scaler_y.fit_transform(vazao_data)
        
        X, y = [], []
        for i in range(n_steps, len(scaled_data)):
            X.append(scaled_data[i-n_steps:i, 0])
            y.append(scaled_data[i, 0])
        
        X = np.array(X).reshape((len(X), n_steps, 1))
        y = np.array(y)
        indices = list(range(n_steps, len(scaled_data)))
    
    if len(X) == 0:
        return {}, None, []
    
    # Split
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Treinar e avaliar
    metrics, models = train_ensemble_and_evaluate(
        X_train, y_train, X_test, y_test, model_type, params, scaler_y
    )
    
    # Predições completas
    predictions = []
    for model in models:
        pred = model.predict(X, verbose=0).flatten()
        predictions.append(pred)
    
    ensemble_pred = np.mean(predictions, axis=0)
    pred_denorm = scaler_y.inverse_transform(ensemble_pred.reshape(-1, 1)).flatten()
    
    return metrics, pred_denorm, indices


def main_advanced(imputed_folder, non_imputed_folder, n_steps=12, use_multivariate=True):
    """
    Função principal com análises avançadas
    """
    print("\n" + "="*80)
    print("ANÁLISE AVANÇADA: IMPACTO DA IMPUTAÇÃO NA PREDIÇÃO")
    print("="*80)
    print(f"Modo: {'MULTIVARIADO' if use_multivariate else 'UNIVARIADO'}")
    print(f"Timesteps: {n_steps}")
    print("="*80 + "\n")
    
    # Configuração
    param_grid = {
        'units': [64, 128],
        'dropout': [0.2, 0.3],
        'learning_rate': [0.001, 0.0005],
    }
    
    matched_files = find_matching_files(imputed_folder, non_imputed_folder)
    
    if not matched_files:
        print("❌ Nenhum par de arquivos encontrado!")
        return pd.DataFrame()
    
    all_results = []
    detailed_results = []
    
    for link, files in matched_files.items():
        print(f"\n{'='*60}")
        print(f"📊 Processando: {link}")
        print(f"{'='*60}")
        
        # Buscar melhores params
        best_params = find_best_params_advanced(files['imputed'], n_steps, param_grid, use_multivariate)
        
        if not best_params:
            print(f"  ❌ Falha ao encontrar parâmetros")
            continue
        
        for model_type in ['lstm', 'gru']:
            print(f"\n  🔹 Modelo: {model_type.upper()}")
            
            # Avaliar com imputação
            print(f"    → Com imputação...")
            metrics_imp, pred_imp, idx_imp = evaluate_advanced(
                files['imputed'], n_steps, model_type, best_params, True, use_multivariate
            )
            
            # Avaliar sem imputação
            print(f"    → Sem imputação...")
            metrics_non, pred_non, idx_non = evaluate_advanced(
                files['non_imputed'], n_steps, model_type, best_params, False, use_multivariate
            )
            
            if not metrics_imp or not metrics_non:
                continue
            
            # Comparar estatisticamente (se possível)
            stat_comparison = {}
            if len(pred_imp) > 0 and len(pred_non) > 0:
                # Calcular erros
                df_imp = pd.read_csv(files['imputed'])
                df_non = pd.read_csv(files['non_imputed'])
                df_non = df_non[df_non['Vazao_BBR'] != -1]
                
                if len(idx_imp) > 0 and len(idx_non) > 0:
                    true_imp = df_imp.iloc[idx_imp]['Vazao_BBR'].values
                    true_non = df_non.iloc[idx_non]['Vazao_BBR'].values
                    
                    errors_imp = true_imp - pred_imp
                    errors_non = true_non - pred_non
                    
                    min_len = min(len(errors_imp), len(errors_non))
                    if min_len > 10:
                        stat_comparison = compare_predictions_statistically(
                            errors_imp[:min_len], errors_non[:min_len]
                        )
            
            # Resultado resumido
            result = {
                'Link': link,
                'Model': model_type.upper(),
                'RMSE_Imputed': metrics_imp.get('RMSE', float('inf')),
                'RMSE_Non_Imputed': metrics_non.get('RMSE', float('inf')),
                'MAE_Imputed': metrics_imp.get('MAE', float('inf')),
                'MAE_Non_Imputed': metrics_non.get('MAE', float('inf')),
                'R2_Imputed': metrics_imp.get('R2', 0),
                'R2_Non_Imputed': metrics_non.get('R2', 0),
                'MAPE_Imputed': metrics_imp.get('MAPE', float('inf')),
                'MAPE_Non_Imputed': metrics_non.get('MAPE', float('inf')),
                'Direction_Acc_Imputed': metrics_imp.get('Direction_Accuracy', 0),
                'Direction_Acc_Non_Imputed': metrics_non.get('Direction_Accuracy', 0),
                'Pearson_Imputed': metrics_imp.get('Pearson_Corr', 0),
                'Pearson_Non_Imputed': metrics_non.get('Pearson_Corr', 0),
                'Statistically_Significant': stat_comparison.get('significantly_better', False),
                'Wilcoxon_P': stat_comparison.get('wilcoxon_pvalue', 1.0)
            }
            
            # Calcular melhorias
            if result['RMSE_Non_Imputed'] != float('inf') and result['RMSE_Imputed'] != float('inf'):
                result['RMSE_Improvement_%'] = ((result['RMSE_Non_Imputed'] - result['RMSE_Imputed']) / result['RMSE_Non_Imputed']) * 100
            else:
                result['RMSE_Improvement_%'] = 0
            
            if result['MAE_Non_Imputed'] != float('inf') and result['MAE_Imputed'] != float('inf'):
                result['MAE_Improvement_%'] = ((result['MAE_Non_Imputed'] - result['MAE_Imputed']) / result['MAE_Non_Imputed']) * 100
            else:
                result['MAE_Improvement_%'] = 0
            
            result['R2_Improvement'] = result['R2_Imputed'] - result['R2_Non_Imputed']
            result['Direction_Improvement_%'] = result['Direction_Acc_Imputed'] - result['Direction_Acc_Non_Imputed']
            
            all_results.append(result)
            
            # Resultado detalhado
            detailed = {
                'Link': link,
                'Model': model_type.upper(),
                'Metrics_Imputed': metrics_imp,
                'Metrics_Non_Imputed': metrics_non,
                'Statistical_Comparison': stat_comparison,
                'Predictions_Imputed': pred_imp,
                'Predictions_Non_Imputed': pred_non,
                'Indices_Imputed': idx_imp,
                'Indices_Non_Imputed': idx_non
            }
            detailed_results.append(detailed)
            
            # Print resumo
            print(f"\n    📈 Resultados:")
            print(f"       RMSE: {result['RMSE_Imputed']:.4f} (imputed) vs {result['RMSE_Non_Imputed']:.4f} (non-imputed)")
            print(f"       MAE:  {result['MAE_Imputed']:.4f} (imputed) vs {result['MAE_Non_Imputed']:.4f} (non-imputed)")
            print(f"       R²:   {result['R2_Imputed']:.4f} (imputed) vs {result['R2_Non_Imputed']:.4f} (non-imputed)")
            print(f"       Direction Acc: {result['Direction_Acc_Imputed']:.2f}% vs {result['Direction_Acc_Non_Imputed']:.2f}%")
            
            if stat_comparison.get('significantly_better', False):
                print(f"       ✓ Diferença ESTATISTICAMENTE SIGNIFICATIVA (p={stat_comparison.get('wilcoxon_pvalue', 1):.4f})")
            
            # Análise de regimes
            regime_imp = metrics_imp.get('regime_analysis', {})
            regime_non = metrics_non.get('regime_analysis', {})
            
            if regime_imp and regime_non:
                print(f"\n    📊 Análise por Regime de Vazão:")
                for regime in ['Low', 'Mid', 'High']:
                    if regime in regime_imp and regime in regime_non:
                        print(f"       {regime}: RMSE {regime_imp[regime]['RMSE']:.4f} vs {regime_non[regime]['RMSE']:.4f}")
            
            # Estabilidade temporal
            temp_imp = metrics_imp.get('temporal_stability', {})
            temp_non = metrics_non.get('temporal_stability', {})
            
            if temp_imp and temp_non:
                print(f"\n    ⏱️  Estabilidade Temporal:")
                print(f"       RMSE std: {temp_imp.get('RMSE_std', 0):.4f} (imputed) vs {temp_non.get('RMSE_std', 0):.4f} (non-imputed)")
                print(f"       R² médio: {temp_imp.get('R2_mean', 0):.4f} (imputed) vs {temp_non.get('R2_mean', 0):.4f} (non-imputed)")
    
    # Criar DataFrame de resultados
    results_df = pd.DataFrame(all_results)
    
    if results_df.empty:
        print("\n❌ Nenhum resultado válido gerado!")
        return results_df, detailed_results
    
    # Filtrar resultados válidos
    valid_mask = (
        (results_df['RMSE_Imputed'] != float('inf')) & 
        (results_df['RMSE_Non_Imputed'] != float('inf'))
    )
    valid_results = results_df[valid_mask].copy()
    
    # Salvar resultados
    output_file = 'resultados_analise_avancada.csv'
    valid_results.to_csv(output_file, index=False)
    print(f"\n💾 Resultados salvos em: {output_file}")
    
    # Salvar análises detalhadas
    save_detailed_analysis(detailed_results, valid_results)
    
    # Mostrar resumo estatístico
    print_statistical_summary(valid_results)
    
    # Gerar visualizações
    generate_visualizations(valid_results, detailed_results)
    
    return valid_results, detailed_results


def save_detailed_analysis(detailed_results, summary_df):
    """
    Salva análises detalhadas em arquivos separados
    """
    output_folder = 'detailed_analysis'
    os.makedirs(output_folder, exist_ok=True)
    
    for detail in detailed_results:
        link = detail['Link']
        model = detail['Model']
        
        # Salvar predições
        df_pred = pd.DataFrame({
            'Index_Imputed': detail['Indices_Imputed'],
            'Prediction_Imputed': detail['Predictions_Imputed']
        })
        
        df_pred.to_csv(os.path.join(output_folder, f'{link}_{model}_predictions.csv'), index=False)
        
        # Salvar métricas completas
        metrics_imp = detail['Metrics_Imputed']
        metrics_non = detail['Metrics_Non_Imputed']
        
        metrics_comparison = {
            'Metric': [],
            'Imputed': [],
            'Non_Imputed': [],
            'Difference': []
        }
        
        for key in metrics_imp.keys():
            if key not in ['temporal_stability', 'regime_analysis', 'autocorr_analysis']:
                imp_val = metrics_imp.get(key, 0)
                non_val = metrics_non.get(key, 0)
                
                if isinstance(imp_val, (int, float)) and isinstance(non_val, (int, float)):
                    metrics_comparison['Metric'].append(key)
                    metrics_comparison['Imputed'].append(imp_val)
                    metrics_comparison['Non_Imputed'].append(non_val)
                    metrics_comparison['Difference'].append(imp_val - non_val)
        
        df_metrics = pd.DataFrame(metrics_comparison)
        df_metrics.to_csv(os.path.join(output_folder, f'{link}_{model}_metrics.csv'), index=False)
    
    print(f"💾 Análises detalhadas salvas em: {output_folder}/")


def print_statistical_summary(results_df):
    """
    Imprime resumo estatístico dos resultados
    """
    print("\n" + "="*80)
    print("📊 RESUMO ESTATÍSTICO COMPLETO")
    print("="*80)
    
    if len(results_df) == 0:
        print("Nenhum resultado válido para análise")
        return
    
    print(f"\n🔢 Total de experimentos válidos: {len(results_df)}")
    
    # Análise geral
    print(f"\n📈 MÉTRICAS GERAIS:")
    print(f"   RMSE:")
    print(f"      Melhoria média: {results_df['RMSE_Improvement_%'].mean():.2f}%")
    print(f"      Melhoria mediana: {results_df['RMSE_Improvement_%'].median():.2f}%")
    print(f"      Casos com melhoria: {len(results_df[results_df['RMSE_Improvement_%'] > 0])}/{len(results_df)}")
    
    print(f"\n   MAE:")
    print(f"      Melhoria média: {results_df['MAE_Improvement_%'].mean():.2f}%")
    print(f"      Melhoria mediana: {results_df['MAE_Improvement_%'].median():.2f}%")
    
    print(f"\n   R²:")
    print(f"      Melhoria média: {results_df['R2_Improvement'].mean():.4f}")
    print(f"      Casos com R² > 0.5 (imputed): {len(results_df[results_df['R2_Imputed'] > 0.5])}/{len(results_df)}")
    
    print(f"\n   Acurácia de Direção:")
    print(f"      Melhoria média: {results_df['Direction_Improvement_%'].mean():.2f}%")
    print(f"      Acurácia média (imputed): {results_df['Direction_Acc_Imputed'].mean():.2f}%")
    
    # Significância estatística
    sig_results = results_df[results_df['Statistically_Significant'] == True]
    print(f"\n📊 SIGNIFICÂNCIA ESTATÍSTICA:")
    print(f"   Casos estatisticamente significativos: {len(sig_results)}/{len(results_df)}")
    if len(sig_results) > 0:
        print(f"   Melhoria média (casos significativos): {sig_results['RMSE_Improvement_%'].mean():.2f}%")
    
    # Por modelo
    print(f"\n🤖 ANÁLISE POR MODELO:")
    for model in results_df['Model'].unique():
        model_results = results_df[results_df['Model'] == model]
        print(f"\n   {model}:")
        print(f"      Total: {len(model_results)} experimentos")
        print(f"      RMSE melhoria média: {model_results['RMSE_Improvement_%'].mean():.2f}%")
        print(f"      R² médio (imputed): {model_results['R2_Imputed'].mean():.4f}")
        print(f"      Acurácia direção média: {model_results['Direction_Acc_Imputed'].mean():.2f}%")
    
    # Melhores e piores casos
    print(f"\n🏆 TOP 5 MELHORES CASOS (por RMSE):")
    top5 = results_df.nlargest(5, 'RMSE_Improvement_%')[['Link', 'Model', 'RMSE_Improvement_%', 'R2_Imputed']]
    print(top5.to_string(index=False))
    
    print(f"\n⚠️  TOP 5 PIORES CASOS (por RMSE):")
    bottom5 = results_df.nsmallest(5, 'RMSE_Improvement_%')[['Link', 'Model', 'RMSE_Improvement_%', 'R2_Imputed']]
    print(bottom5.to_string(index=False))
    
    # Distribuição das melhorias
    print(f"\n📊 DISTRIBUIÇÃO DAS MELHORIAS:")
    bins = [float('-inf'), -10, 0, 10, 20, float('inf')]
    labels = ['<-10%', '-10~0%', '0~10%', '10~20%', '>20%']
    results_df['Improvement_Range'] = pd.cut(results_df['RMSE_Improvement_%'], bins=bins, labels=labels)
    print(results_df['Improvement_Range'].value_counts().sort_index())


def generate_visualizations(results_df, detailed_results):
    """
    Gera visualizações dos resultados
    """
    if len(results_df) == 0:
        return
    
    output_folder = 'visualizations'
    os.makedirs(output_folder, exist_ok=True)
    
    # 1. Comparação RMSE
    plt.figure(figsize=(12, 6))
    x = np.arange(len(results_df))
    width = 0.35
    
    plt.bar(x - width/2, results_df['RMSE_Imputed'], width, label='With Imputation', alpha=0.8)
    plt.bar(x + width/2, results_df['RMSE_Non_Imputed'], width, label='Without Imputation', alpha=0.8)
    
    plt.xlabel('Experiment')
    plt.ylabel('RMSE')
    plt.title('RMSE Comparison: With vs Without Imputation')
    plt.legend()
    plt.xticks(x, [f"{row['Link'][:10]}\n{row['Model']}" for _, row in results_df.iterrows()], rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'rmse_comparison.png'), dpi=300)
    plt.close()
    
    # 2. Distribuição de melhorias
    plt.figure(figsize=(10, 6))
    plt.hist(results_df['RMSE_Improvement_%'], bins=20, edgecolor='black', alpha=0.7)
    plt.axvline(0, color='red', linestyle='--', label='No improvement')
    plt.axvline(results_df['RMSE_Improvement_%'].mean(), color='green', linestyle='--', label='Mean improvement')
    plt.xlabel('RMSE Improvement (%)')
    plt.ylabel('Frequency')
    plt.title('Distribution of RMSE Improvements')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'improvement_distribution.png'), dpi=300)
    plt.close()
    
    # 3. R² comparison
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df['R2_Non_Imputed'], results_df['R2_Imputed'], alpha=0.6, s=100)
    plt.plot([0, 1], [0, 1], 'r--', label='No change')
    plt.xlabel('R² Without Imputation')
    plt.ylabel('R² With Imputation')
    plt.title('R² Score Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'r2_comparison.png'), dpi=300)
    plt.close()
    
    # 4. Direction Accuracy
    plt.figure(figsize=(10, 6))
    results_df_sorted = results_df.sort_values('Direction_Acc_Imputed', ascending=False)
    x = np.arange(len(results_df_sorted))
    
    plt.plot(x, results_df_sorted['Direction_Acc_Imputed'], marker='o', label='With Imputation', linewidth=2)
    plt.plot(x, results_df_sorted['Direction_Acc_Non_Imputed'], marker='s', label='Without Imputation', linewidth=2)
    plt.xlabel('Experiment (sorted by imputed accuracy)')
    plt.ylabel('Direction Accuracy (%)')
    plt.title('Direction Prediction Accuracy Comparison')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_folder, 'direction_accuracy.png'), dpi=300)
    plt.close()
    
    # 5. Heatmap de métricas
    metrics_cols = ['RMSE_Improvement_%', 'MAE_Improvement_%', 'R2_Improvement', 'Direction_Improvement_%']
    available_metrics = [col for col in metrics_cols if col in results_df.columns]
    
    if len(available_metrics) > 0:
        plt.figure(figsize=(12, 8))
        heatmap_data = results_df[['Link', 'Model'] + available_metrics].set_index(['Link', 'Model'])
        sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='RdYlGn', center=0, cbar_kws={'label': 'Improvement'})
        plt.title('Improvement Metrics Heatmap')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder, 'metrics_heatmap.png'), dpi=300)
        plt.close()
    
    print(f"📊 Visualizações salvas em: {output_folder}/")


# ============================================================================
# EXECUÇÃO PRINCIPAL
# ============================================================================

if __name__ == "__main__":
    # Configurar caminhos
    IMPUTED_FOLDER = "../../datasets/multivariada-imputed-selecionados"
    NON_IMPUTED_FOLDER = "../../datasets/multivariada-post-process"
    
    # Executar análise avançada
    print("\n" + "🚀"*40)
    print("INICIANDO ANÁLISE AVANÇADA DE PREDIÇÃO")
    print("🚀"*40)
    
    # Tentar modo multivariado primeiro
    print("\n📊 Tentando modo MULTIVARIADO...")
    results_multi, detailed_multi = main_advanced(
        IMPUTED_FOLDER, 
        NON_IMPUTED_FOLDER, 
        n_steps=12, 
        use_multivariate=True
    )
    
    # Se não funcionar, tentar univariado
    if results_multi.empty:
        print("\n⚠️  Modo multivariado falhou. Tentando UNIVARIADO...")
        results_uni, detailed_uni = main_advanced(
            IMPUTED_FOLDER, 
            NON_IMPUTED_FOLDER, 
            n_steps=12, 
            use_multivariate=False
        )
        results = results_uni
        detailed = detailed_uni
    else:
        results = results_multi
        detailed = detailed_multi
    
    print("\n" + "✅"*40)
    print("ANÁLISE CONCLUÍDA COM SUCESSO!")
    print("✅"*40 + "\n")

2025-11-05 16:55:27.303876: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-11-05 16:55:27.313845: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-05 16:55:27.668438: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-05 16:55:29.226868: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,


🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀
INICIANDO ANÁLISE AVANÇADA DE PREDIÇÃO
🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀🚀

📊 Tentando modo MULTIVARIADO...

ANÁLISE AVANÇADA: IMPACTO DA IMPUTAÇÃO NA PREDIÇÃO
Modo: MULTIVARIADO
Timesteps: 12

✓ Encontrados 406 pares de arquivos

📊 Processando: ap-ba
  Buscando melhores hiperparâmetros...


2025-11-05 16:55:30.254678: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


  ❌ Falha ao encontrar parâmetros

📊 Processando: ce-am
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: pa-pi
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: ce-pa
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: rj-to
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: rr-pb
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: am-mt
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: go-ce
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: sc-se
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: rs-rj
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: pe-ac
  Buscando melhores hiperparâmetros...
  ❌ Falha ao encontrar parâmetros

📊 Processando: 